In [ ]:
import os


os.chdir("../")


os.getcwd()


'd:\\DA_workspace\\alzheimers_prediction\\src'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    PredefinedSplit,
    GridSearchCV,
)
from classifier.Mylib import myfuncs
from sklearn.decomposition import KernelPCA
import plotly.express as px
from sklearn import metrics
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.cluster import KMeans


In [ ]:
train_features = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/train_features.pkl"
)
train_features_double = pd.concat([train_features, train_features], axis=0)
spliter = PredefinedSplit(
    test_fold=[-1] * len(train_features) + [0] * len(train_features)
)


def get_silhouette_score(estimator, X, y=None):
    labels = estimator.predict(X)
    return metrics.silhouette_score(X, labels)


param_grid = {"n_clusters": range(90, 100 + 1)}
kmeans = KMeans(n_init=10, random_state=42)
gs = GridSearchCV(
    kmeans,
    param_grid,
    cv=spliter,
    scoring=lambda estimator, X: get_silhouette_score(estimator, X),
    verbose=2,
)

gs.fit(train_features_double)

gs.best_params_


Fitting 1 folds for each of 11 candidates, totalling 11 fits
[CV] END ......................................n_clusters=90; total time=  45.7s


KeyboardInterrupt: 

In [ ]:
importance["feature"].tolist()[:3]


['do_get_Genetic_Risk_Factor', 'do_Family_History', 'Age']

In [ ]:
df.shape


(74283, 25)

In [ ]:
df_train, df_test = train_test_split(
    df, stratify=df["Alzheimer’s Diagnosis"], test_size=0.1, random_state=42
)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


data_folder = "../artifacts/data_ingestion"
os.makedirs(data_folder, exist_ok=True)

myfuncs.save_python_object(os.path.join(data_folder, "raw_data.pkl"), df)
myfuncs.save_python_object(os.path.join(data_folder, "train_data.pkl"), df_train)
myfuncs.save_python_object(os.path.join(data_folder, "test_data.pkl"), df_test)
